## 학습목표
 1. dataframe 제공 함수를 통한 데이터 처리 기법 숙지
 2. groupby, concat, merge, pivot, pivot_table, stack, unstack, transorm 등

### group by
  + 아래의 세 단계를 포함 (SQL의 group by 와 개념적으로는 동일, 사용법은 유사)
    - 데이터 분할
    - operation 적용
    - 데이터 병합

# a['a'] is a[0] why False?

In [1]:
import pandas as pd
import numpy as np

In [2]:
a = pd.Series(range(4), index=['a', 'b', 'c', 'd'])

In [3]:
# 1955년부터의 유로리그 우승팀 데이터
df = pd.read_csv('./data/euro_winners.csv')

In [4]:
df.head(15)

,Season,Nation,Winners,Score,Runners-up,Runner-UpNation,Venue,Attendance
0,1955–56,Spain,Real Madrid,4–3,Stade de Reims,France,"Parc des Princes,Paris",38239
1,1956–57,Spain,Real Madrid,2–0,Fiorentina,Italy,"Santiago Bernabéu Stadium, Madrid",124000
2,1957–58,Spain,Real Madrid,3–2,Milan,Italy,"Heysel Stadium,Brussels",67000
3,1958–59,Spain,Real Madrid,2–0,Stade de Reims,France,"Neckarstadion,Stuttgart",72000
4,1959–60,Spain,Real Madrid,7–3,Eintracht Frankfurt,Germany,"Hampden Park,Glasgow",127621
5,1960–61,Portugal,Benfica,3–2,Barcelona,Spain,"Wankdorf Stadium,Bern",26732
6,1961–62,Portugal,Benfica,5–3,Real Madrid,Spain,"Olympisch Stadion,Amsterdam",61257
7,1962–63,Italy,Milan,2–1,Benfica,Portugal,"Wembley Stadium,London",45715
8,1963–64,Italy,Internazionale,3–1,Real Madrid,Spain,"Prater Stadium,Vienna",71333
9,1964–65,Italy,Internazionale,1–0,Benfica,Portugal,"San Siro, Milan",89000


In [5]:
# 국가별로 그룹핑 
nation_group = df.groupby('Nation')
type(nation_group)

pandas.core.groupby.DataFrameGroupBy

In [6]:
nation_group.groups # groups 속성으로 group 정보를 dict로 반환

{'England': Int64Index([12, 21, 22, 23, 24, 25, 26, 28, 43, 49, 52, 56], dtype='int64'),
 'France': Int64Index([37], dtype='int64'),
 'Germany': Int64Index([18, 19, 20, 27, 41, 45, 57], dtype='int64'),
 'Italy': Int64Index([7, 8, 9, 13, 29, 33, 34, 38, 40, 47, 51, 54], dtype='int64'),
 'Netherlands': Int64Index([14, 15, 16, 17, 32, 39], dtype='int64'),
 'Portugal': Int64Index([5, 6, 31, 48], dtype='int64'),
 'Romania': Int64Index([30], dtype='int64'),
 'Scotland': Int64Index([11], dtype='int64'),
 'Spain': Int64Index([0, 1, 2, 3, 4, 10, 36, 42, 44, 46, 50, 53, 55], dtype='int64'),
 'Yugoslavia': Int64Index([35], dtype='int64')}

In [7]:
len(nation_group.groups) # 그룹의 개수

10

* group 함수
 - count, size 등

In [8]:
nation_group.count()

,Season,Winners,Score,Runners-up,Runner-UpNation,Venue,Attendance
Nation,,,,,,,
England,12,12,12,12,12,12,12
France,1,1,1,1,1,1,1
Germany,7,7,7,7,7,7,7
Italy,12,12,12,12,12,12,12
Netherlands,6,6,6,6,6,6,6
Portugal,4,4,4,4,4,4,4
Romania,1,1,1,1,1,1,1
Scotland,1,1,1,1,1,1,1
Spain,13,13,13,13,13,13,13


In [9]:
nation_group.size()

Nation
England        12
France          1
Germany         7
Italy          12
Netherlands     6
Portugal        4
Romania         1
Scotland        1
Spain          13
Yugoslavia      1
dtype: int64

In [10]:
nation_group.size().sort_values(ascending = False)

Nation
Spain          13
Italy          12
England        12
Germany         7
Netherlands     6
Portugal        4
Yugoslavia      1
Scotland        1
Romania         1
France          1
dtype: int64

In [11]:
# 2개 이상의 컬럼으로 grouping 가능
club_group = df.groupby(['Nation', 'Winners'])
club_group.size().sort_values(ascending = False)

Nation       Winners          
Spain        Real Madrid          9
Italy        Milan                7
Germany      Bayern Munich        5
England      Liverpool            5
Spain        Barcelona            4
Netherlands  Ajax                 4
England      Manchester United    3
Italy        Internazionale       3
             Juventus             2
Portugal     Porto                2
             Benfica              2
England      Nottingham Forest    2
             Chelsea              1
France       Marseille            1
Yugoslavia   Red Star Belgrade    1
Germany      Borussia Dortmund    1
             Hamburg              1
Netherlands  Feyenoord            1
             PSV Eindhoven        1
Romania      Steaua Bucure?ti     1
Scotland     Celtic               1
England      Aston Villa          1
dtype: int64

* 각 리그별 경기수, 득점 데이터

In [42]:
df = pd.read_csv('./data/goal_stats_euro_leagues_2012-13.csv')

In [43]:
df.head(20)

,Month,Stat,EPL,La Liga,Serie A,Bundesliga
0,08/01/2012,MatchesPlayed,20.0,20,10.0,10.0
1,09/01/2012,MatchesPlayed,38.0,39,50.0,44.0
2,10/01/2012,MatchesPlayed,31.0,31,39.0,27.0
3,11/01/2012,MatchesPlayed,50.0,41,42.0,46.0
4,12/01/2012,MatchesPlayed,59.0,39,39.0,26.0
5,01/01/2013,MatchesPlayed,42.0,40,40.0,18.0
6,02/01/2013,MatchesPlayed,30.0,40,40.0,36.0
7,03/01/2013,MatchesPlayed,35.0,38,39.0,36.0
8,04/01/2013,MatchesPlayed,42.0,42,41.0,36.0
9,05/01/2013,MatchesPlayed,33.0,40,40.0,27.0


* inplace parameter
 - 대부분의 dataframe 연산은 원본데이터를 유지하고 복사본을 반환
 - 원본 자체를 변경하고 싶으면 inplace paramter를 True로 설정

In [44]:
df.head()

,Month,Stat,EPL,La Liga,Serie A,Bundesliga
0,08/01/2012,MatchesPlayed,20.0,20,10.0,10.0
1,09/01/2012,MatchesPlayed,38.0,39,50.0,44.0
2,10/01/2012,MatchesPlayed,31.0,31,39.0,27.0
3,11/01/2012,MatchesPlayed,50.0,41,42.0,46.0
4,12/01/2012,MatchesPlayed,59.0,39,39.0,26.0


In [45]:
# Month column을 index로 생성
df.set_index('Month')

# 혹은 아래와 같이 가능
#df.set_index(['Month', 'Stat'], inplace=True)

,Stat,EPL,La Liga,Serie A,Bundesliga
Month,,,,,
08/01/2012,MatchesPlayed,20.0,20,10.0,10.0
09/01/2012,MatchesPlayed,38.0,39,50.0,44.0
10/01/2012,MatchesPlayed,31.0,31,39.0,27.0
11/01/2012,MatchesPlayed,50.0,41,42.0,46.0
12/01/2012,MatchesPlayed,59.0,39,39.0,26.0
01/01/2013,MatchesPlayed,42.0,40,40.0,18.0
02/01/2013,MatchesPlayed,30.0,40,40.0,36.0
03/01/2013,MatchesPlayed,35.0,38,39.0,36.0
04/01/2013,MatchesPlayed,42.0,42,41.0,36.0


In [37]:
# Month column을 index로 생성
df.set_index('Month', inplace=False)

# 혹은 아래와 같이 가능
#df.set_index(['Month', 'Stat'], inplace=True)

,Stat,EPL,La Liga,Serie A,Bundesliga
Month,,,,,
08/01/2012,MatchesPlayed,20.0,20,10.0,10.0
09/01/2012,MatchesPlayed,38.0,39,50.0,44.0
10/01/2012,MatchesPlayed,31.0,31,39.0,27.0
11/01/2012,MatchesPlayed,50.0,41,42.0,46.0
12/01/2012,MatchesPlayed,59.0,39,39.0,26.0
01/01/2013,MatchesPlayed,42.0,40,40.0,18.0
02/01/2013,MatchesPlayed,30.0,40,40.0,36.0
03/01/2013,MatchesPlayed,35.0,38,39.0,36.0
04/01/2013,MatchesPlayed,42.0,42,41.0,36.0


In [46]:
df.head()

,Month,Stat,EPL,La Liga,Serie A,Bundesliga
0,08/01/2012,MatchesPlayed,20.0,20,10.0,10.0
1,09/01/2012,MatchesPlayed,38.0,39,50.0,44.0
2,10/01/2012,MatchesPlayed,31.0,31,39.0,27.0
3,11/01/2012,MatchesPlayed,50.0,41,42.0,46.0
4,12/01/2012,MatchesPlayed,59.0,39,39.0,26.0


In [47]:
# 다시 index를 column 레벨로 변경
df = df.reset_index()

In [48]:
df.head(100)

,index,Month,Stat,EPL,La Liga,Serie A,Bundesliga
0,0,08/01/2012,MatchesPlayed,20.0,20,10.0,10.0
1,1,09/01/2012,MatchesPlayed,38.0,39,50.0,44.0
2,2,10/01/2012,MatchesPlayed,31.0,31,39.0,27.0
3,3,11/01/2012,MatchesPlayed,50.0,41,42.0,46.0
4,4,12/01/2012,MatchesPlayed,59.0,39,39.0,26.0
5,5,01/01/2013,MatchesPlayed,42.0,40,40.0,18.0
6,6,02/01/2013,MatchesPlayed,30.0,40,40.0,36.0
7,7,03/01/2013,MatchesPlayed,35.0,38,39.0,36.0
8,8,04/01/2013,MatchesPlayed,42.0,42,41.0,36.0
9,9,05/01/2013,MatchesPlayed,33.0,40,40.0,27.0


* groupby에 함수 사용

In [49]:
df.head()

,index,Month,Stat,EPL,La Liga,Serie A,Bundesliga
0,0,08/01/2012,MatchesPlayed,20.0,20,10.0,10.0
1,1,09/01/2012,MatchesPlayed,38.0,39,50.0,44.0
2,2,10/01/2012,MatchesPlayed,31.0,31,39.0,27.0
3,3,11/01/2012,MatchesPlayed,50.0,41,42.0,46.0
4,4,12/01/2012,MatchesPlayed,59.0,39,39.0,26.0


In [51]:
# group by year
# 함수가 전달되는 경우 함수의 파라미터로 index 값이 전달
year_group = df.groupby(lambda month : month.split('/')[2])

AttributeError: 'int' object has no attribute 'split'

In [ ]:
year_group.groups

In [ ]:
year_group.size()

In [ ]:
# index의 경우에는 column name이 아닌, 레벨로 grouping 가능
month_group = df.groupby(level = 0)

In [ ]:
# 혹은 다음과 같이 index 자체를 명시 가능
month_group = df.groupby(level = 'Month')

In [ ]:
for name, group in month_group:
    print(name, '\n')
    print(group)

### multi index grouping

* index 항목 제거

In [52]:
df.reset_index(inplace=True)

In [53]:
df.head()

,level_0,index,Month,Stat,EPL,La Liga,Serie A,Bundesliga
0,0,0,08/01/2012,MatchesPlayed,20.0,20,10.0,10.0
1,1,1,09/01/2012,MatchesPlayed,38.0,39,50.0,44.0
2,2,2,10/01/2012,MatchesPlayed,31.0,31,39.0,27.0
3,3,3,11/01/2012,MatchesPlayed,50.0,41,42.0,46.0
4,4,4,12/01/2012,MatchesPlayed,59.0,39,39.0,26.0


* 2가지 컬럼으로 인덱싱

In [54]:
# 멀티 레벨 인덱스를 생성
df = df.set_index(['Month', 'Stat'])
df.head()

,,level_0,index,EPL,La Liga,Serie A,Bundesliga
Month,Stat,,,,,,
08/01/2012,MatchesPlayed,0,0,20.0,20,10.0,10.0
09/01/2012,MatchesPlayed,1,1,38.0,39,50.0,44.0
10/01/2012,MatchesPlayed,2,2,31.0,31,39.0,27.0
11/01/2012,MatchesPlayed,3,3,50.0,41,42.0,46.0
12/01/2012,MatchesPlayed,4,4,59.0,39,39.0,26.0


* 멀티 인덱스에서 row 선택

In [55]:
df.loc['08/01/2012']

,level_0,index,EPL,La Liga,Serie A,Bundesliga
Stat,,,,,,
MatchesPlayed,0,0,20.0,20,10.0,10.0
GoalsScored,11,11,57.0,60,21.0,23.0


In [56]:
df.loc['08/01/2012'].loc['MatchesPlayed']

level_0        0.0
index          0.0
EPL           20.0
La Liga       20.0
Serie A       10.0
Bundesliga    10.0
Name: MatchesPlayed, dtype: float64

In [57]:
df.loc['08/01/2012', 'MatchesPlayed']

level_0        0.0
index          0.0
EPL           20.0
La Liga       20.0
Serie A       10.0
Bundesliga    10.0
Name: (08/01/2012, MatchesPlayed), dtype: float64

* 멀티 인덱스의 레벨을 이용한 그룹핑

In [58]:
# multi level grouping
month_stat_group = df.groupby(level = [0, 1])

In [59]:
for name, group in month_stat_group:
    print(name) # tuple
    print(group)
    print()

('01/01/2013', 'GoalsScored')
                        level_0  index    EPL  La Liga  Serie A  Bundesliga
Month      Stat                                                            
01/01/2013 GoalsScored       16     16  117.0      121    104.0        51.0

('01/01/2013', 'MatchesPlayed')
                          level_0  index   EPL  La Liga  Serie A  Bundesliga
Month      Stat                                                             
01/01/2013 MatchesPlayed        5      5  42.0       40     40.0        18.0

('02/01/2013', 'GoalsScored')
                        level_0  index   EPL  La Liga  Serie A  Bundesliga
Month      Stat                                                           
02/01/2013 GoalsScored       17     17  87.0      110    100.0       101.0

('02/01/2013', 'MatchesPlayed')
                          level_0  index   EPL  La Liga  Serie A  Bundesliga
Month      Stat                                                             
02/01/2013 MatchesPlayed        6  

In [60]:
month_stat_group = df.groupby(level = ['Month', 'Stat'])

In [61]:
for name, group in month_stat_group:
    print(name) # tuple
    print(group)

('01/01/2013', 'GoalsScored')
                        level_0  index    EPL  La Liga  Serie A  Bundesliga
Month      Stat                                                            
01/01/2013 GoalsScored       16     16  117.0      121    104.0        51.0
('01/01/2013', 'MatchesPlayed')
                          level_0  index   EPL  La Liga  Serie A  Bundesliga
Month      Stat                                                             
01/01/2013 MatchesPlayed        5      5  42.0       40     40.0        18.0
('02/01/2013', 'GoalsScored')
                        level_0  index   EPL  La Liga  Serie A  Bundesliga
Month      Stat                                                           
02/01/2013 GoalsScored       17     17  87.0      110    100.0       101.0
('02/01/2013', 'MatchesPlayed')
                          level_0  index   EPL  La Liga  Serie A  Bundesliga
Month      Stat                                                             
02/01/2013 MatchesPlayed        6     

* row 선택하기

In [62]:
df2 = pd.read_csv('./data/goal_stats_euro_leagues_2012-13.csv')

In [63]:
df2.head()

,Month,Stat,EPL,La Liga,Serie A,Bundesliga
0,08/01/2012,MatchesPlayed,20.0,20,10.0,10.0
1,09/01/2012,MatchesPlayed,38.0,39,50.0,44.0
2,10/01/2012,MatchesPlayed,31.0,31,39.0,27.0
3,11/01/2012,MatchesPlayed,50.0,41,42.0,46.0
4,12/01/2012,MatchesPlayed,59.0,39,39.0,26.0


In [64]:
df2.loc[0] # 0은 인덱스를 의미, 여기서는 단순한 순서

Month            08/01/2012
Stat          MatchesPlayed
EPL                      20
La Liga                  20
Serie A                  10
Bundesliga               10
Name: 0, dtype: object

In [65]:
df2 = df2.set_index('Month')
df2.head()

,Stat,EPL,La Liga,Serie A,Bundesliga
Month,,,,,
08/01/2012,MatchesPlayed,20.0,20,10.0,10.0
09/01/2012,MatchesPlayed,38.0,39,50.0,44.0
10/01/2012,MatchesPlayed,31.0,31,39.0,27.0
11/01/2012,MatchesPlayed,50.0,41,42.0,46.0
12/01/2012,MatchesPlayed,59.0,39,39.0,26.0


In [66]:
# index가 Month로 바뀌었으므로
df2.loc['08/01/2012']

,Stat,EPL,La Liga,Serie A,Bundesliga
Month,,,,,
08/01/2012,MatchesPlayed,20.0,20,10.0,10.0
08/01/2012,GoalsScored,57.0,60,21.0,23.0


In [67]:
# 0번째
df2.iloc[0]

Stat          MatchesPlayed
EPL                      20
La Liga                  20
Serie A                  10
Bundesliga               10
Name: 08/01/2012, dtype: object

In [68]:
df2 = df2.reset_index()

* multiple index

In [69]:
df2 = df2.set_index(['Month', 'Stat'])
df2.head()

,,EPL,La Liga,Serie A,Bundesliga
Month,Stat,,,,
08/01/2012,MatchesPlayed,20.0,20,10.0,10.0
09/01/2012,MatchesPlayed,38.0,39,50.0,44.0
10/01/2012,MatchesPlayed,31.0,31,39.0,27.0
11/01/2012,MatchesPlayed,50.0,41,42.0,46.0
12/01/2012,MatchesPlayed,59.0,39,39.0,26.0


In [70]:
df2.iloc[0]

EPL           20.0
La Liga       20.0
Serie A       10.0
Bundesliga    10.0
Name: (08/01/2012, MatchesPlayed), dtype: float64

In [71]:
# multi index의 경우 ','로 구별하여 인덱싱 가능
df2.loc['08/01/2012', 'MatchesPlayed']

EPL           20.0
La Liga       20.0
Serie A       10.0
Bundesliga    10.0
Name: (08/01/2012, MatchesPlayed), dtype: float64

* grouping with multi index

In [72]:
df2.head()

,,EPL,La Liga,Serie A,Bundesliga
Month,Stat,,,,
08/01/2012,MatchesPlayed,20.0,20,10.0,10.0
09/01/2012,MatchesPlayed,38.0,39,50.0,44.0
10/01/2012,MatchesPlayed,31.0,31,39.0,27.0
11/01/2012,MatchesPlayed,50.0,41,42.0,46.0
12/01/2012,MatchesPlayed,59.0,39,39.0,26.0


In [73]:
stat_group = df2.groupby(level = 'Stat')
for name, group in stat_group:
    print(name)
    print(group)

GoalsScored
                          EPL  La Liga  Serie A  Bundesliga
Month      Stat                                            
08/01/2012 GoalsScored   57.0       60     21.0        23.0
09/01/2012 GoalsScored  111.0      112    133.0       135.0
10/01/2012 GoalsScored   95.0       88     97.0        77.0
11/01/2012 GoalsScored  121.0      116    120.0       137.0
12/01/2012 GoalsScored  183.0      109    125.0        72.0
01/01/2013 GoalsScored  117.0      121    104.0        51.0
02/01/2013 GoalsScored   87.0      110    100.0       101.0
03/01/2013 GoalsScored   91.0      101     99.0       106.0
04/01/2013 GoalsScored  105.0      127    102.0       104.0
05/01/2013 GoalsScored   96.0      109    102.0        92.0
06/01/2013 GoalsScored    NaN       80      NaN         NaN
MatchesPlayed
                           EPL  La Liga  Serie A  Bundesliga
Month      Stat                                             
08/01/2012 MatchesPlayed  20.0       20     10.0        10.0
09/01/2012 

In [74]:
# group의 sum method를 이용
# 각 리그당 경기 수와 골을 나타냄
stat_group.sum()

,EPL,La Liga,Serie A,Bundesliga
Stat,,,,
GoalsScored,1063.0,1133,1003.0,898.0
MatchesPlayed,380.0,380,380.0,306.0


In [75]:
# 혹은 level을 이용하여 dataframe level에서 계산 가능
df2.sum(level = 'Stat')

,EPL,La Liga,Serie A,Bundesliga
Stat,,,,
GoalsScored,1063.0,1133,1003.0,898.0
MatchesPlayed,380.0,380,380.0,306.0


In [76]:
total_goal_df = stat_group.sum()
total_goal_df.head()

,EPL,La Liga,Serie A,Bundesliga
Stat,,,,
GoalsScored,1063.0,1133,1003.0,898.0
MatchesPlayed,380.0,380,380.0,306.0


###  각 리그의 경기당 평균골 구하기

* 평균 골 Series 생성

In [ ]:
goals_per_game_df = total_goal_df.loc['GoalsScored'] \
                        / total_goal_df.loc['MatchesPlayed']
goals_per_game_df

* dataframe으로 변경

In [ ]:
pd.DataFrame(goals_per_game_df)

* Transpose하여 row, column 변경

In [ ]:
goals_per_game_df = pd.DataFrame(goals_per_game_df).T
goals_per_game_df

* mapping dictionary로 인덱스 값 변경

In [ ]:
goals_per_game_df = goals_per_game_df \
                        .rename(index = \
                                {0 : 'goals per game'})
goals_per_game_df

* 새로운 row로 추가

In [ ]:
total_goal_df.append(goals_per_game_df)

* 소수점 2자리까지 출력하도록 설정

In [ ]:
pd.options.display.float_format='{:.2f}'.format
total_goal_df.append(goals_per_game_df)

### Formating
  - https://pyformat.info/ 참고

### aggregate method
 - groupby 객체에 aggregate(집합) 함수 적용하여 그룹별 데이터 확인 가능

In [ ]:
print(stat_group)
for name, group in stat_group:
    print(name)
    print(group)

In [ ]:
stat_group.sum()

In [ ]:
stat_group.aggregate(len)

In [ ]:
stat_group.aggregate(np.sum)

* multiple function 적용
  - 복수개의 column 레벨로 데이터 생성

In [ ]:
stat_group.aggregate([np.sum, np.mean, np.size])

### transform 함수

* 가상 sales data 실습
 - excel로 구성된 가상의 판매 데이터 이용하여 transformation 실습
 - excel 데이터를 읽기 위해서 xlrd 모듈이 필요
   - pip install xlrd

In [ ]:
df = pd.read_excel("data/sales_transactions.xlsx")

In [ ]:
df.head(13)

In [ ]:
df.groupby('order').sum()

In [ ]:
# ext price 컬럼만 sum
df.groupby('order')["ext price"].sum()

* 위의 정보를 다시 원래의 frame에 합치려면?
 - 아래와 같이 다시 merge 하여 합칠 수도 있음
 - 가능은 하지만, 더 쉽게 transform으로 가능

In [ ]:
order_total = df.groupby('order')["ext price"].sum().rename("total").reset_index()
order_total

In [ ]:
result = df.merge(order_total, on='order')
result["portion"] = result["ext price"] / result["total"]

In [ ]:
result

* **transform 함수 사용!**
 - group by 후 transform을 사용하면 원본의 row를 유지한 상태에서 transform에 전달된 함수를 호출하여 aggregation
 - row의 개수가 유지되므로 바로 cloumn 추가 방법을 사용하여 원본 dataframe에 추가 가능

In [ ]:
df.groupby('order')["ext price"].sum()

In [ ]:
df.groupby('order')["ext price"].transform(np.sum)

In [ ]:
df['total'] = df.groupby('order')["ext price"].transform(np.sum)
df["portion"] = df["ext price"] / df["total"]
df

# < 9/27 >

###  pivot 
 - dataframe의 형태를 변경
 - 변경 시, index, column, data를 명시

* ** 식물실험 데이터**
 - ctrl, tr1, tr2 환경에서의 회차별 weight 관측 데이터

In [2]:
plant_df = pd.read_csv('data/PlantGrowth.csv')
plant_df.head(30)

,observation,weight,group
0,1,4.17,ctrl
1,2,5.58,ctrl
2,3,5.18,ctrl
3,4,6.11,ctrl
4,5,4.50,ctrl
5,6,4.61,ctrl
6,7,5.17,ctrl
7,8,4.53,ctrl
8,9,5.33,ctrl
9,10,5.14,ctrl


In [3]:
plant_df
# control(ctrl)하여 실험한 값과, 두 개의 다른 조건(tr1, tr2)을 주었을 때의 실험에 대한 결과 
# data가 stacked 됨. (말 그대로 쌓여있음)
# 데이터 분석하기 편리하지 않음

,observation,weight,group
0,1,4.17,ctrl
1,2,5.58,ctrl
2,3,5.18,ctrl
3,4,6.11,ctrl
4,5,4.50,ctrl
5,6,4.61,ctrl
6,7,5.17,ctrl
7,8,4.53,ctrl
8,9,5.33,ctrl
9,10,5.14,ctrl


In [4]:
# 아래와 같이 filtering하여 각각의 결과를 뽑을 수 있으나, pivot, unstack을 이용하여 분석하기 용이하도록 변형 가능
plant_df[plant_df['group'] == 'ctrl']   #boolean으로 불러옴

,observation,weight,group
0,1,4.17,ctrl
1,2,5.58,ctrl
2,3,5.18,ctrl
3,4,6.11,ctrl
4,5,4.50,ctrl
5,6,4.61,ctrl
6,7,5.17,ctrl
7,8,4.53,ctrl
8,9,5.33,ctrl
9,10,5.14,ctrl


In [5]:
# dataframe reshaping
# 각각 index, columns, values로 명시하여 data frame의 shape을 변경 가능
plant_df.pivot(index = 'observation', 
               columns = 'group', 
               values = 'weight')

#각각의 그룹들이 Column으로 올라감

group,ctrl,trt1,trt2
observation,,,
1,4.17,4.81,6.31
2,5.58,4.17,5.12
3,5.18,4.41,5.54
4,6.11,3.59,5.50
5,4.50,5.87,5.37
6,4.61,3.83,5.29
7,5.17,6.03,4.92
8,4.53,4.89,6.15
9,5.33,4.32,5.80


In [6]:
# pivot_table 으로도 동일 결과 가능
pd.pivot_table(plant_df, values = 'weight', index = 'observation', columns='group')

#동시에 여러개가 올 때 피봇테이블을 쓴다.

group,ctrl,trt1,trt2
observation,,,
1,4.17,4.81,6.31
2,5.58,4.17,5.12
3,5.18,4.41,5.54
4,6.11,3.59,5.50
5,4.50,5.87,5.37
6,4.61,3.83,5.29
7,5.17,6.03,4.92
8,4.53,4.89,6.15
9,5.33,4.32,5.80


In [7]:
pd.pivot_table(plant_df, values = 'weight', columns='group')

#각 값들을 합하여 평균을 낸 것.

group,ctrl,trt1,trt2
weight,5.032,4.661,5.526


In [8]:
pd.pivot_table(plant_df, values = 'weight', columns='observation')

#값들이 동시에 여러개가 올 때 유용하다.

observation,1,2,3,4,5,6,7,8,9,10
weight,5.096667,4.956667,5.043333,5.066667,5.246667,4.576667,5.373333,5.19,5.15,5.03


* **pivot, pivot_table의 차이**
 - pivot_table 
   - pivot 결과 생성 시, 값 선택이 vague한 경우, aggfunc 사용하여 채움


In [9]:
from collections import OrderedDict

table = OrderedDict((
    ("Item", ['Item0', 'Item0', 'Item1', 'Item1']),
    ('CType',['Gold', 'Bronze', 'Gold', 'Silver']),
    ('USD',  [1, 2, 3, 4]),
    ('EU',   [1, 2, 3, 4])
))
df = pd.DataFrame(table)

In [10]:
df

,Item,CType,USD,EU
0,Item0,Gold,1,1
1,Item0,Bronze,2,2
2,Item1,Gold,3,3
3,Item1,Silver,4,4


In [11]:
df.pivot(index='Item', columns='CType', values='USD')

#인덱스를 아이템으로... 
#아이템1에 브론즈가 없으므로 nan

CType,Bronze,Gold,Silver
Item,,,
Item0,2.0,1.0,NaN
Item1,NaN,3.0,4.0


In [12]:
df.pivot(index='Item', columns='CType')

USD                 EU            
CType Bronze Gold Silver Bronze Gold Silver
Item                                       
Item0    2.0  1.0    NaN    2.0  1.0    NaN
Item1    NaN  3.0    4.0    NaN  3.0    4.0

In [13]:
table = OrderedDict((
    ("Item", ['Item0', 'Item0', 'Item0', 'Item1']),
    ('CType',['Gold', 'Bronze', 'Gold', 'Silver']),
    ('USD',  [1, 2, 3, 4]),
    ('EU',   [1, 2, 3, 4])
))

df = pd.DataFrame(table)

df

,Item,CType,USD,EU
0,Item0,Gold,1,1
1,Item0,Bronze,2,2
2,Item0,Gold,3,3
3,Item1,Silver,4,4


In [14]:
# Item0과 Gold에 대해서 1, 3중 어떤 것을 채워야 할지 알 수 없음  /// 에러발생
df.pivot(index='Item', columns='CType', values='USD')

#복수개의 값을 가져오지 못하기 때문에 pivot은 에러

ValueError: Index contains duplicate entries, cannot reshape

In [15]:
df.pivot_table(index='Item', 
               columns='CType', 
               values='USD')

# 1과 3을 평균냈으므로, 값이 2가 됨.

CType,Bronze,Gold,Silver
Item,,,
Item0,2.0,2.0,NaN
Item1,NaN,NaN,4.0


In [17]:
import numpy as np
df.pivot_table(index='Item', 
               columns='CType', 
               values='USD', aggfunc = np.sum)

CType,Bronze,Gold,Silver
Item,,,
Item0,2.0,4.0,NaN
Item1,NaN,NaN,4.0


###  stack & unstack
 - stack : inner-most column -> inner-most index
 - unstack : inner-most index -> inner-most column
 
 - 둘은 역의 관계에 있음
 
 > row와 column을 바꾸는 것

In [18]:
plant_df

,observation,weight,group
0,1,4.17,ctrl
1,2,5.58,ctrl
2,3,5.18,ctrl
3,4,6.11,ctrl
4,5,4.50,ctrl
5,6,4.61,ctrl
6,7,5.17,ctrl
7,8,4.53,ctrl
8,9,5.33,ctrl
9,10,5.14,ctrl


In [19]:
stacked = plant_df.set_index(['group', 'observation'])   #인덱스 2개 생성(첫번째 인덱스, 두번째 인덱스...)
stacked

weight
group observation        
ctrl  1              4.17
      2              5.58
      3              5.18
      4              6.11
      5              4.50
      6              4.61
      7              5.17
      8              4.53
      9              5.33
      10             5.14
trt1  1              4.81
      2              4.17
      3              4.41
      4              3.59
      5              5.87
      6              3.83
      7              6.03
      8              4.89
      9              4.32
      10             4.69
trt2  1              6.31
      2              5.12
      3              5.54
      4              5.50
      5              5.37
      6              5.29
      7              4.92
      8              6.15
      9              5.80
      10             5.26

In [20]:
# 가장 바깥 레벨의 row -> column으로 이동
stacked.unstack()

#observation을 column으로 바꿈

weight                                                      
observation     1     2     3     4     5     6     7     8     9     10
group                                                                   
ctrl          4.17  5.58  5.18  6.11  4.50  4.61  5.17  4.53  5.33  5.14
trt1          4.81  4.17  4.41  3.59  5.87  3.83  6.03  4.89  4.32  4.69
trt2          6.31  5.12  5.54  5.50  5.37  5.29  4.92  6.15  5.80  5.26

In [21]:
# 레벨 0의 row -> column으로 이동
stacked.unstack(0)                      #첫번째 레벨을 column으로!!

# 레벨의 순서가 아닌, 네임으로도 명시 가능
# stacked.unstack(level = 'group')

weight            
group         ctrl  trt1  trt2
observation                   
1             4.17  4.81  6.31
2             5.58  4.17  5.12
3             5.18  4.41  5.54
4             6.11  3.59  5.50
5             4.50  5.87  5.37
6             4.61  3.83  5.29
7             5.17  6.03  4.92
8             4.53  4.89  6.15
9             5.33  4.32  5.80
10            5.14  4.69  5.26

In [27]:
# 레벨 0의 row -> column으로 이동
stacked.unstack(1)                      #두번째 레벨을 column으로!!

# 레벨의 순서가 아닌, 네임으로도 명시 가능
# stacked.unstack(level = 'group')

weight                                                      
observation     1     2     3     4     5     6     7     8     9     10
group                                                                   
ctrl          4.17  5.58  5.18  6.11  4.50  4.61  5.17  4.53  5.33  5.14
trt1          4.81  4.17  4.41  3.59  5.87  3.83  6.03  4.89  4.32  4.69
trt2          6.31  5.12  5.54  5.50  5.37  5.29  4.92  6.15  5.80  5.26

In [22]:
# 컬럼을 다시 가장 하위의 row로 이동
stacked.unstack(0).stack()           #다시 stack을 하면, 2번째 레벨의 혹은 가장 하위의 레벨로 들어가게 됨.

weight
observation group        
1           ctrl     4.17
            trt1     4.81
            trt2     6.31
2           ctrl     5.58
            trt1     4.17
            trt2     5.12
3           ctrl     5.18
            trt1     4.41
            trt2     5.54
4           ctrl     6.11
            trt1     3.59
            trt2     5.50
5           ctrl     4.50
            trt1     5.87
            trt2     5.37
6           ctrl     4.61
            trt1     3.83
            trt2     5.29
7           ctrl     5.17
            trt1     6.03
            trt2     4.92
8           ctrl     4.53
            trt1     4.89
            trt2     6.15
9           ctrl     5.33
            trt1     4.32
            trt2     5.80
10          ctrl     5.14
            trt1     4.69
            trt2     5.26

### ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ

###  Docstring

In [23]:
def add(a, b, c):
    return a+b+c

In [ ]:
add()  #이 상태에서 Shift + Tab을 누르면 Docstring이 없음

In [26]:
def add(a, b, c):
    '''
    calculation the summation of three numeric values
    
    Parameter:
      a, b, c - numeric types
    
    Return:
      summation of a, b and c
    
    Exceptions:
      if c is negative, exception would be raised
      
    '''
    
    if c < 0:
        raise Exception('invalid values')
        
    return a+b+c

In [ ]:
add()    #다시 Shift + Tab을 누르면 Docstring이 생김.

### ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ

### Concat
> 따로 떨어져 있는 리스트를 아래로 이어붙이거나 옆으로 이어붙일 수 있음

In [28]:
import pandas as pd
import numpy as np

In [29]:
df1 = pd.DataFrame({'key1' : np.arange(10), 'value1' : np.random.randn(10)})

In [30]:
df1

,key1,value1
0,0,-0.387518
1,1,0.388263
2,2,1.196511
3,3,-1.648488
4,4,-0.158568
5,5,1.069364
6,6,-0.466647
7,7,1.039616
8,8,0.637368
9,9,-0.541899


In [31]:
df2 = pd.DataFrame({'key1' : np.arange(10), 'value1' : np.random.randn(10)})

In [32]:
df2

,key1,value1
0,0,-1.859847
1,1,0.996631
2,2,-1.329884
3,3,0.314832
4,4,-0.255066
5,5,-0.761715
6,6,0.520321
7,7,0.429972
8,8,-0.828723
9,9,-0.473076


In [33]:
# axis=0 기본적으로 row를 따라 합침 // 아래로 합친다
pd.concat([df1, df2])

,key1,value1
0,0,-0.387518
1,1,0.388263
2,2,1.196511
3,3,-1.648488
4,4,-0.158568
5,5,1.069364
6,6,-0.466647
7,7,1.039616
8,8,0.637368
9,9,-0.541899


In [37]:
# axis=1로 주어 column을 따라 합치는 것도 가능
pd.concat([df1, df2], axis=1)

# 이건 옆으로 이어붙인 것.

,key1,value1,key1,value1
0,0,-0.387518,0,-1.859847
1,1,0.388263,1,0.996631
2,2,1.196511,2,-1.329884
3,3,-1.648488,3,0.314832
4,4,-0.158568,4,-0.255066
5,5,1.069364,5,-0.761715
6,6,-0.466647,6,0.520321
7,7,1.039616,7,0.429972
8,8,0.637368,8,-0.828723
9,9,-0.541899,9,-0.473076


In [36]:
# axis=0 기본적으로 row를 따라 합침
# 이때 기존의 인덱스를 무시하려면 ignore_index 파라미터를 True로 변경
pd.concat([df1, df2], ignore_index=True)

#기존의 인덱스를 무시하고 이어붙인 순서대로 인덱스를 이어나간다.

,key1,value1
0,0,-0.387518
1,1,0.388263
2,2,1.196511
3,3,-1.648488
4,4,-0.158568
5,5,1.069364
6,6,-0.466647
7,7,1.039616
8,8,0.637368
9,9,-0.541899


* column 네임이 다를 경우

In [38]:
df3 = pd.DataFrame({'key2' : np.arange(10), 'value2' : np.random.randn(10)})

In [40]:
# 없는 컬럼에 대해서는 nan으로 채움
pd.concat([df1, df3], ignore_index=True)

# 모든 컬럼을 다 가져오는데 없는 것은 그냥 nan 처리를 함.

,key1,key2,value1,value2
0,0.0,NaN,-0.387518,NaN
1,1.0,NaN,0.388263,NaN
2,2.0,NaN,1.196511,NaN
3,3.0,NaN,-1.648488,NaN
4,4.0,NaN,-0.158568,NaN
5,5.0,NaN,1.069364,NaN
6,6.0,NaN,-0.466647,NaN
7,7.0,NaN,1.039616,NaN
8,8.0,NaN,0.637368,NaN
9,9.0,NaN,-0.541899,NaN


* index를 설정하고 concat

In [41]:
df11 = df1.set_index('key1')

In [42]:
df22 = df2.set_index('key1')

In [43]:
pd.concat([df11, df22])

# 이건 그냥 행으로 내려버린 것

,value1
key1,
0,-0.387518
1,0.388263
2,1.196511
3,-1.648488
4,-0.158568
5,1.069364
6,-0.466647
7,1.039616
8,0.637368


In [44]:
result = pd.concat([df11, df22], axis=1)
result

# 이건 그냥 열로 이어붙인 것.

,value1,value1
key1,,
0,-0.387518,-1.859847
1,0.388263,0.996631
2,1.196511,-1.329884
3,-1.648488,0.314832
4,-0.158568,-0.255066
5,1.069364,-0.761715
6,-0.466647,0.520321
7,1.039616,0.429972
8,0.637368,-0.828723


### Merge & Join

In [45]:
df1 = pd.DataFrame({'key1' : np.arange(8), 
                    'key2' : ['LA', 'LA', 'SF', 'SF', 'SF', 'NY', 'NY', 'LV'], 
                    'value1' : np.random.randn(8)})

In [46]:
df1

,key1,key2,value1
0,0,LA,-0.555242
1,1,LA,-0.912727
2,2,SF,0.592560
3,3,SF,-2.511289
4,4,SF,-1.877310
5,5,NY,-0.931068
6,6,NY,0.783787
7,7,LV,-0.185031


In [47]:
df2 = pd.DataFrame({'key1' : [1, 1, 2, 2, 2, 3, 3], 
                    'key2' : ['LA', 'LA', 'KC', 'KC', 'NY', 'SF', 'SF'], 
                    'value1' : np.random.randn(7)})

In [48]:
df2

,key1,key2,value1
0,1,LA,0.547300
1,1,LA,-0.899403
2,2,KC,0.423247
3,2,KC,-0.678192
4,2,NY,1.025804
5,3,SF,-0.260631
6,3,SF,-1.617026


* on
 - join 하고자 하는 대상이 되는 column 명시

In [49]:
pd.merge(df1, df2, on='key1')

,key1,key2_x,value1_x,key2_y,value1_y
0,1,LA,-0.912727,LA,0.547300
1,1,LA,-0.912727,LA,-0.899403
2,2,SF,0.592560,KC,0.423247
3,2,SF,0.592560,KC,-0.678192
4,2,SF,0.592560,NY,1.025804
5,3,SF,-2.511289,SF,-0.260631
6,3,SF,-2.511289,SF,-1.617026


In [50]:
pd.merge(df1, df2, on='key2')   #문자열이 같은것끼리 묶음

,key1_x,key2,value1_x,key1_y,value1_y
0,0,LA,-0.555242,1,0.547300
1,0,LA,-0.555242,1,-0.899403
2,1,LA,-0.912727,1,0.547300
3,1,LA,-0.912727,1,-0.899403
4,2,SF,0.592560,3,-0.260631
5,2,SF,0.592560,3,-1.617026
6,3,SF,-2.511289,3,-0.260631
7,3,SF,-2.511289,3,-1.617026
8,4,SF,-1.877310,3,-0.260631
9,4,SF,-1.877310,3,-1.617026


In [52]:
pd.merge(df1, df2, on='key2', how = 'left')

,key1_x,key2,value1_x,key1_y,value1_y
0,0,LA,-0.555242,1.0,0.547300
1,0,LA,-0.555242,1.0,-0.899403
2,1,LA,-0.912727,1.0,0.547300
3,1,LA,-0.912727,1.0,-0.899403
4,2,SF,0.592560,3.0,-0.260631
5,2,SF,0.592560,3.0,-1.617026
6,3,SF,-2.511289,3.0,-0.260631
7,3,SF,-2.511289,3.0,-1.617026
8,4,SF,-1.877310,3.0,-0.260631
9,4,SF,-1.877310,3.0,-1.617026


* how parameter
 - SQL의 join과 같이 동작
 - inner : 기본 값, 일치 하는 값들로만 조인
 - left : left outer join과 같이 동작
 - right : right outer join과 같이 동작
 - outer : full outer join과 같이 동작, 즉 left + right

* left join
 - 왼쪽 df 기준으로 join, 없는 경우 nan 할당

In [53]:
pd.merge(df1, df2, on='key2', how='left')

,key1_x,key2,value1_x,key1_y,value1_y
0,0,LA,-0.555242,1.0,0.547300
1,0,LA,-0.555242,1.0,-0.899403
2,1,LA,-0.912727,1.0,0.547300
3,1,LA,-0.912727,1.0,-0.899403
4,2,SF,0.592560,3.0,-0.260631
5,2,SF,0.592560,3.0,-1.617026
6,3,SF,-2.511289,3.0,-0.260631
7,3,SF,-2.511289,3.0,-1.617026
8,4,SF,-1.877310,3.0,-0.260631
9,4,SF,-1.877310,3.0,-1.617026


In [54]:
pd.merge(df2, df1, on='key2', how='left')  #위와의 차이점은 df1과 2의 순서만 바꾼 것. // how를 left에서 right로 바꿔도 됨

,key1_x,key2,value1_x,key1_y,value1_y
0,1,LA,0.547300,0.0,-0.555242
1,1,LA,0.547300,1.0,-0.912727
2,1,LA,-0.899403,0.0,-0.555242
3,1,LA,-0.899403,1.0,-0.912727
4,2,KC,0.423247,NaN,NaN
5,2,KC,-0.678192,NaN,NaN
6,2,NY,1.025804,5.0,-0.931068
7,2,NY,1.025804,6.0,0.783787
8,3,SF,-0.260631,2.0,0.592560
9,3,SF,-0.260631,3.0,-2.511289


* right join
 - 오른쪽 df 기준으로 join, 없는 경우 nan 할당

In [55]:
pd.merge(df1, df2, on='key2', how='right')

,key1_x,key2,value1_x,key1_y,value1_y
0,0.0,LA,-0.555242,1,0.547300
1,1.0,LA,-0.912727,1,0.547300
2,0.0,LA,-0.555242,1,-0.899403
3,1.0,LA,-0.912727,1,-0.899403
4,2.0,SF,0.592560,3,-0.260631
5,3.0,SF,-2.511289,3,-0.260631
6,4.0,SF,-1.877310,3,-0.260631
7,2.0,SF,0.592560,3,-1.617026
8,3.0,SF,-2.511289,3,-1.617026
9,4.0,SF,-1.877310,3,-1.617026


* full join
 - left join 과 right join union

In [56]:
pd.merge(df1, df2, on='key2', how='outer')

,key1_x,key2,value1_x,key1_y,value1_y
0,0.0,LA,-0.555242,1.0,0.547300
1,0.0,LA,-0.555242,1.0,-0.899403
2,1.0,LA,-0.912727,1.0,0.547300
3,1.0,LA,-0.912727,1.0,-0.899403
4,2.0,SF,0.592560,3.0,-0.260631
5,2.0,SF,0.592560,3.0,-1.617026
6,3.0,SF,-2.511289,3.0,-0.260631
7,3.0,SF,-2.511289,3.0,-1.617026
8,4.0,SF,-1.877310,3.0,-0.260631
9,4.0,SF,-1.877310,3.0,-1.617026


* **multi key 조인**
 - join 하고자 할 column 리스트로 구성

In [57]:
pd.merge(df1, df2, on=['key1', 'key2'])   # 대괄로 리스트로 하면 같은 것끼리만 묶여서 출력됨

,key1,key2,value1_x,value1_y
0,1,LA,-0.912727,0.547300
1,1,LA,-0.912727,-0.899403
2,3,SF,-2.511289,-0.260631
3,3,SF,-2.511289,-1.617026


In [58]:
pd.merge(df1, df2, on=['key1', 'key2'], how='left')

# 중복되는 sufficies는 underbar로 바뀐다.

,key1,key2,value1_x,value1_y
0,0,LA,-0.555242,NaN
1,1,LA,-0.912727,0.547300
2,1,LA,-0.912727,-0.899403
3,2,SF,0.592560,NaN
4,3,SF,-2.511289,-0.260631
5,3,SF,-2.511289,-1.617026
6,4,SF,-1.877310,NaN
7,5,NY,-0.931068,NaN
8,6,NY,0.783787,NaN
9,7,LV,-0.185031,NaN


In [61]:
pd.merge(df1, df2, on=['key1', 'key2'], how='left')

# 중복되는 sufficies는 underbar로 바뀐다.

,key1,key2,value1_x,value1_y
0,0,LA,-0.555242,NaN
1,1,LA,-0.912727,0.547300
2,1,LA,-0.912727,-0.899403
3,2,SF,0.592560,NaN
4,3,SF,-2.511289,-0.260631
5,3,SF,-2.511289,-1.617026
6,4,SF,-1.877310,NaN
7,5,NY,-0.931068,NaN
8,6,NY,0.783787,NaN
9,7,LV,-0.185031,NaN


In [62]:
# 나머지 column의 이름이 겹칠 경우 suffix 지정 가능
pd.merge(df1, df2, on=['key1', 'key2'], how='left', 
         suffixes=('_left', '_right'))

,key1,key2,value1_left,value1_right
0,0,LA,-0.555242,NaN
1,1,LA,-0.912727,0.547300
2,1,LA,-0.912727,-0.899403
3,2,SF,0.592560,NaN
4,3,SF,-2.511289,-0.260631
5,3,SF,-2.511289,-1.617026
6,4,SF,-1.877310,NaN
7,5,NY,-0.931068,NaN
8,6,NY,0.783787,NaN
9,7,LV,-0.185031,NaN


* index가 있는 경우 merge
 - left_index : 왼쪽 프레임에서 인덱스를 사용할 경우 True 명시
 - right_index : 오른쪽 프레임에서 인덱스를 사용할 경우 True 명시

In [64]:
df11 = df1.set_index('key1')
df11

#key1을 내려버림(행의 형태로)

,key2,value1
key1,,
0,LA,-0.555242
1,LA,-0.912727
2,SF,0.592560
3,SF,-2.511289
4,SF,-1.877310
5,NY,-0.931068
6,NY,0.783787
7,LV,-0.185031


In [65]:
df22 = df2.set_index('key1')
df22

,key2,value1
key1,,
1,LA,0.547300
1,LA,-0.899403
2,KC,0.423247
2,KC,-0.678192
2,NY,1.025804
3,SF,-0.260631
3,SF,-1.617026


In [66]:
# index끼리 조인할 것이라고 명시 가능
pd.merge(df11, df22, left_index=True, right_index=True)

,key2_x,value1_x,key2_y,value1_y
key1,,,,
1,LA,-0.912727,LA,0.547300
1,LA,-0.912727,LA,-0.899403
2,SF,0.592560,KC,0.423247
2,SF,0.592560,KC,-0.678192
2,SF,0.592560,NY,1.025804
3,SF,-2.511289,SF,-0.260631
3,SF,-2.511289,SF,-1.617026


In [68]:
pd.merge(df1, df22, left_on='key1', right_index=True)

,key1,key2_x,value1_x,key2_y,value1_y
1,1,LA,-0.912727,LA,0.547300
1,1,LA,-0.912727,LA,-0.899403
2,2,SF,0.592560,KC,0.423247
2,2,SF,0.592560,KC,-0.678192
2,2,SF,0.592560,NY,1.025804
3,3,SF,-2.511289,SF,-0.260631
3,3,SF,-2.511289,SF,-1.617026
